In [1]:
import pandas as pd
import numpy as np

In [20]:
# Part 1

In [2]:
df1 = pd.read_csv("data/users_with_age_data.csv")

In [3]:
df2 = pd.read_csv("data/users_with_gender_data.csv")

In [4]:
df1.merge(df2, on="user_id", how="left")

,user_id,age,gender
0,1,30,'M'
1,2,40,'F'
2,4,14,'F'
3,6,35,'M'
4,7,16,NaN
5,8,50,NaN


In [7]:
users = df1.merge(df2, on="user_id", how="outer")

In [21]:
users['gender'] = users['gender'].fillna("'F'")

In [22]:
users['age'] = users['age'].fillna(users['age'].mean())

In [8]:
users

,user_id,age,gender
0,1,30.000000,'M'
1,2,40.000000,'F'
2,4,14.000000,'F'
3,6,35.000000,'M'
4,7,16.000000,'F'
5,8,50.000000,'F'
6,3,30.833333,'M'
7,5,30.833333,'M'


In [14]:
grouped = users.groupby('gender')

In [15]:
grouped.count() # aggregation

,user_id,age
gender,,
'F',4,4
'M',4,4


In [16]:
grouped['age'].mean() # get mean age of each group

gender
'F'    30.000000
'M'    31.666667
Name: age, dtype: float64

In [17]:
users['minor'] = users['age'].apply(lambda x: True if x < 18 else False)

#def func(x):
#    if x < 18:
#        return True
#    return False
#
#users['minor'] = users['age'].apply(func)

In [18]:
users

,user_id,age,gender,minor
0,1,30.000000,'M',False
1,2,40.000000,'F',False
2,4,14.000000,'F',True
3,6,35.000000,'M',False
4,7,16.000000,'F',True
5,8,50.000000,'F',False
6,3,30.833333,'M',False
7,5,30.833333,'M',False


In [19]:
# Part 2

In [23]:
users = pd.read_csv('movie/users.dat', delimiter='::', names=['user_id','gender','age','occupation_code','zip'], engine='python')

In [24]:
ratings = pd.read_csv('movie/ratings.dat', delimiter='::', names=['user_id','movie_id','rating','timestamp'], engine='python')

In [8]:
movies = pd.read_csv('movie/movies.dat', delimiter='::', names=['movie_id','title','genre'], engine='python')

In [30]:
# joins the all the files
merged = users.merge(ratings, on="user_id").merge(movies, on="movie_id")

In [31]:
sorted_movies = merged.groupby("title").count().sort_values(by='rating', ascending=False)

In [32]:
# displays 5 movies with most # of ratings
sorted_movies.head(5)

,user_id,gender,age,occupation_code,zip,movie_id,rating,timestamp,genre
title,,,,,,,,,
American Beauty (1999),3428,3428,3428,3428,3428,3428,3428,3428,3428
Star Wars: Episode IV - A New Hope (1977),2991,2991,2991,2991,2991,2991,2991,2991,2991
Star Wars: Episode V - The Empire Strikes Back (1980),2990,2990,2990,2990,2990,2990,2990,2990,2990
Star Wars: Episode VI - Return of the Jedi (1983),2883,2883,2883,2883,2883,2883,2883,2883,2883
Jurassic Park (1993),2672,2672,2672,2672,2672,2672,2672,2672,2672


In [37]:
# list moviess with >= 250 ratings
active_titles = sorted_movies.query('rating >= 250')
# Alternatively:
# active_titles = top_movies[top_movies['rating'] >= 250]

In [54]:
ungrouped_sorted_movies = merged[merged['title'].isin(active_titles.index)]

In [58]:
separated_rating = ungrouped_sorted_movies.groupby(['gender','title'], as_index=False).agg({'rating':np.mean})

In [66]:
# movies with highest rating for male & female ?
separated_rating.sort_values(['gender','rating'],ascending=False).head(3)

,gender,title,rating
1676,M,"Godfather, The (1972)",4.583333
2175,M,Seven Samurai (The Magnificent Seven) (Shichin...,4.576628
2187,M,"Shawshank Redemption, The (1994)",4.560625


In [65]:
# movies with highest rating for male & female
separated_rating.sort_values(['gender','rating'],ascending=False).groupby('gender').head(3)

,gender,title,rating
1676,M,"Godfather, The (1972)",4.583333
2175,M,Seven Samurai (The Magnificent Seven) (Shichin...,4.576628
2187,M,"Shawshank Redemption, The (1994)",4.560625
233,F,"Close Shave, A (1995)",4.644444
1203,F,"Wrong Trousers, The (1993)",4.588235
1064,F,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650


In [70]:
pivoted_ratings = separated_rating.pivot('title', 'gender')

In [75]:
# female rating.mean() - men rating.mean()
# more negative = low rated by woman, highly rated by men
# more positive = highly rated by woman, lowly rated by women 

pivoted_ratings['rating'].apply(lambda x: x['F'] - x['M'], axis=1).sort_values()

title
Good, The Bad and The Ugly, The (1966)        -0.726351
Kentucky Fried Movie, The (1977)              -0.676359
Dumb & Dumber (1994)                          -0.638608
Longest Day, The (1962)                       -0.619682
Cable Guy, The (1996)                         -0.613787
Evil Dead II (Dead By Dawn) (1987)            -0.611985
Hidden, The (1987)                            -0.607167
Rocky III (1982)                              -0.581801
Caddyshack (1980)                             -0.573602
For a Few Dollars More (1965)                 -0.544704
Porky's (1981)                                -0.539489
Animal House (1978)                           -0.538286
Exorcist, The (1973)                          -0.529605
Fright Night (1985)                           -0.526316
Barb Wire (1996)                              -0.515020
Rocky II (1979)                               -0.501515
Beavis and Butt-head Do America (1996)        -0.498054
Big Trouble in Little China (1986)        

In [78]:
# highest std
ratings_std = ungrouped_sorted_movies.groupby('title')['rating'].std()

In [79]:
ratings_std.nlargest(5)

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Name: rating, dtype: float64

In [2]:
import sqlite3

In [3]:
con = sqlite3.connect('flights/flights.db')

In [6]:
df = pd.read_sql('SELECT * from airlines', con)

In [7]:
df

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N
5,5,6,223 Flight Unit State Airline,\N,None,CHD,CHKALOVSK-AVIA,Russia,N
6,6,7,224th Flight Unit,\N,None,TTF,CARGO UNIT,Russia,N
7,7,8,247 Jet Ltd,\N,None,TWF,CLOUD RUNNER,United Kingdom,N
8,8,9,3D Aviation,\N,None,SEC,SECUREX,United States,N
9,9,10,40-Mile Air,\N,Q5,MLA,MILE-AIR,United States,Y
